# ANN-Results

Final results of the proposed methodology obtain by running the following code. It is executed 10 ten times to check the stability of the model(you can also use a loop).

Importing libraries

In [1]:
from final_hybrid_csv import CSV_READER
from ann_hybrid import ANN
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

Checking the availablity of the GPU

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available(): 
    print("gpu available")

Setting the optimize parameters that we got in the optimization phase

In [3]:
num_epochs,num_classes,batch_size,learning_rate,w_decay = 10, 9,50,0.001,0.001

Reading the train and test hybrid data

In [5]:
train_path='n_final_hybrid_full_train.csv'
test_path='n_final_hybrid_test.csv'

train_data = CSV_READER(train_path)
test_data = CSV_READER(test_path)
test_size = len(test_data)

train_data_loader = data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
test_data_loader  = data.DataLoader(test_data, batch_size=test_size, shuffle=True) 

Creating the model, defining the loss and optimizer

In [6]:
model=ANN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=w_decay)

Training a model to the defined number of epochs

In [7]:
for epoch in range(num_epochs):

    for i, (images,labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs[:,0].data, 1)
        loss = criterion(outputs[:,0], labels)
        x=np.array((predicted==labels).cpu())
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

Doing the test phase and printing loss and accuracy

In [8]:
for y,(images,labels) in enumerate(test_data_loader):
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    _, predicted = torch.max(outputs[:,0].data, 1)
    test_loss = criterion(outputs[:,0], labels)
    test_x=np.array((predicted==labels).cpu())
    print("Test_loss="+str(test_loss.item())+" Test_acc="+str((sum(test_x))*100/len(test_x)))

Generating the confusion matrix

In [9]:
test_labels_bin=label_binarize(labels.cpu(),classes=[0,1,2,3,4,5,6,7,8])
cm=confusion_matrix(labels.cpu(),predicted.cpu())
print(cm)

Note:- For doing the optimization of ANN we use the following code:




from final_hybrid_csv import CSV_READER
from ann_hybrid import ANN
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available(): 
    print("gpu availible")


num_epochs,num_classes,batch_size,learning_rate = 10, 9,50,0.001


train_path='n_final_hybrid_train.csv'
val_path='n_final_hybrid_valid.csv'




train_data = CSV_READER(train_path)
val_data = CSV_READER(val_path)
val_size = len(val_data)

train_data_loader = data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = data.DataLoader(val_data, batch_size=val_size, shuffle=True) 

model=ANN().to(device)


criterion = nn.CrossEntropyLoss()

path_to_checkpoint='.../_optimize/'

check_name='checkpoint.pth.tar'



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=0.001)


def save_checkpoint(state, is_best, filename,loss):
    if is_best:
        print ("=> Saving a new lowest loss : "+str(loss))
        torch.save(state, filename)  # save checkpoint


total_step = len(train_data_loader)
lowest_loss=0.200
checkpoint_step= int(total_step/8)


for epoch in range(num_epochs):

    for i, (images,labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)
    
        # Forward pass
        outputs = model(images)

        _, predicted = torch.max(outputs[:,0].data, 1)
    
        loss = criterion(outputs[:,0], labels)
 
        x=np.array((predicted==labels).cpu())
        # Backward and optimize
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        val_loss_list=[]
        val_acc_list =[]
        
        for y,(images,labels) in enumerate(val_data_loader):
            images=images.to(device)
            labels=labels.to(device)
            outputs=model(images)
            _, predicted = torch.max(outputs[:,0].data, 1)
            val_loss = criterion(outputs[:,0], labels)
            val_loss_list.append(val_loss.item())
            
            val_x=np.array((predicted==labels).cpu())
            val_acc=(sum(val_x))*100/len(val_x)
            val_acc_list.append(val_acc)
        mean_loss= np.mean(np.array(val_loss_list))
        mean_acc = np.mean(np.array(val_acc_list))
        is_best= bool(mean_loss<lowest_loss)
        
        if(is_best):
            lowest_loss= min(mean_loss,lowest_loss)
            pathe= path_to_checkpoint+str(lowest_loss)+" "+str(epoch+1)+" "+check_name
            save_checkpoint({'epoch':epoch + 1,'state_dict': model.state_dict(),'lowest_loss': lowest_loss }, is_best,pathe,lowest_loss)
            is_best= False
 
        print("Epoch ["+str(epoch+1)+"/"+str(num_epochs)+"],Batch_no["+str(i+1)+"/"+str(total_step)+"] "+"Loss:"+str(format(loss.item(),'.4f'))+" Acc :"+str(format((sum(x))*100/len(x),'.4f'))+" Val_loss:"+str(format(mean_loss,'.4f'))+" Val_acc:"+str(format(mean_acc,'.0f')))
        